<a href="https://colab.research.google.com/github/MANU-CHAUHAN/twitter-disaster-analysis/blob/main/Capstone_BERT_LARGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers -q

In [58]:
import pandas as pd
import re
import string
from transformers import AutoTokenizer, TFBertModel
import numpy as np
from sklearn import model_selection
URL = 'https://drive.google.com/file/d/1fLX0XxkbSXatVevvndyqkBDxNOgWcUHP/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+URL.split('/')[-2]
#df = pd.read_pickle(path)
df = pd.read_csv(path)
df.head()

tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')
bert = TFBertModel.from_pretrained('bert-large-uncased')

Some layers from the model checkpoint at bert-large-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-large-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [73]:
df_c = df.copy()
df_c=df_c.drop(columns=['keyword','id', 'location'])
df_c=df_c.dropna()

In [ ]:
df_c.text=df_c.text.apply(lambda x:x.lower() )
#removing square brackets
df_c.text=df_c.text.apply(lambda x:re.sub('\[.*?\]', '', x) )
df_c.text=df_c.text.apply(lambda x:re.sub('<.*?>+', '', x) )
#removing hyperlink
df_c.text=df_c.text.apply(lambda x:re.sub('https?://\S+|www\.\S+', '', x) )
#removing puncuation
df_c.text=df_c.text.apply(lambda x:re.sub('[%s]' % re.escape(string.punctuation), '', x) )
df_c.text=df_c.text.apply(lambda x:re.sub('\n' , '', x) )
#remove words containing numbers
df_c.text=df_c.text.apply(lambda x:re.sub('\w*\d\w*' , '', x) )

#df_c.iloc[:-10]

In [129]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df_c['text'],df_c['target'],test_size=0.3)

In [101]:
Train_X

8083                          mary coming to troy rescue  
6369     sinjar massacre yazidis blast lack of action o...
7311     nuclear reactor railguns would be a great way ...
526      one direction is my pick for  fan army directi...
3056     ûïlolgop  cases of voter fraud a year we need...
                               ...                        
10273    kasiakosek the drive sucks in my case because ...
2116     iphone twist ultimate preparedness library  pr...
7050                  the ol meltdown victory for the mets
8280     georgous what alternatives legal alternatives ...
7982     the latest more homes razed by northern califo...
Name: text, Length: 5329, dtype: object

In [145]:
 #train
 x_1= tokenizer(
    text=Train_X.to_list(),
    add_special_tokens=False, #only in this case
    max_length=41, #not sure about this
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
 y_1=Train_Y.to_numpy()
# test
 x_2= tokenizer(
    text=Test_X.to_list(),
    add_special_tokens=False, #only in this case
    max_length=41,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
 y_2=Test_Y.to_numpy()

In [146]:
x_1

{'input_ids': <tf.Tensor: shape=(5329, 41), dtype=int32, numpy=
array([[ 2005,  2216,  2040, ...,     0,     0,     0],
       [ 3422,  2023,  3199, ...,     0,     0,     0],
       [ 2525,  6634,  6834, ...,     0,     0,     0],
       ...,
       [ 1045,  2245,  1996, ...,     0,     0,     0],
       [ 6608,  1037,  9964, ...,     0,     0,     0],
       [11652,     0,     0, ...,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(5329, 41), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int32)>}

In [147]:
import tensorflow as tf
from tensorflow import keras
max_len = 41 
input_ids = keras.Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = keras.Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[1]
out = tf.keras.layers.Dropout(0.1)(embeddings)
out = keras.layers.Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = keras.layers.Dense(32,activation = 'relu')(out)
y = keras.layers.Dense(1,activation = 'sigmoid')(out)
    
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [148]:
optimizer = keras.optimizers.Adam(
    learning_rate=6e-06, # this learning rate is taken from huggingface website 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)

# Set loss and metrics
loss = keras.losses.BinaryCrossentropy(from_logits = True)
metric = keras.metrics.BinaryAccuracy('accuracy'),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [ ]:
train_history = model.fit(
    x ={'input_ids':x_1['input_ids'],'attention_mask':x_1['attention_mask']} ,
    y = y_1, epochs=1, batch_size=32
)

 12/167 [=>............................] - ETA: 2:04:33 - loss: 0.7327 - accuracy: 0.5312

In [128]:
predicted = model.predict({'input_ids':x_2['input_ids'],'attention_mask':x_2['attention_mask']})
y_predicted = np.where(predicted>0.5,1,0)
y_predictedd = y_predicted.reshape((1,3263))[0]

ValueError: ignored